<a href="https://colab.research.google.com/github/Kimsongbeen/Photato_photo/blob/main/android_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyngrok
!pip install diffusers==0.2.4
!pip install --upgrade torch torchvision torchaudio pillow

  Using cached diffusers-0.2.4-py3-none-any.whl.metadata (14 kB)
Using cached diffusers-0.2.4-py3-none-any.whl (112 kB)
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.3
    Uninstalling diffusers-0.30.3:
      Successfully uninstalled diffusers-0.30.3


In [ ]:
!pip install --upgrade diffusers

from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16)
pipe.to("cuda")

  Using cached diffusers-0.30.3-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.30.3-py3-none-any.whl (2.7 MB)
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.2.4
    Uninstalling diffusers-0.2.4:
      Successfully uninstalled diffusers-0.2.4


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

TypeError: getattr(): attribute name must be string

In [ ]:
from pyngrok import ngrok

port = 5000

ngrok.set_auth_token('NGROK 인증키')
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

In [ ]:
import os
from io import BytesIO

from flask import Flask, request, jsonify, send_file

app = Flask(__name__)

@app.route('/')
def home():
    return "Flask server is running!"

@app.route('/generate', methods=['POST'])
def generate_image():
    data = request.get_json()
    text = data.get('text', '')

    if not text:
        return jsonify({'error': 'Text is required'}), 400

    print(f"Received text: {text}")

    try:
      image = pipe(text).images[0]
      img_io = BytesIO()
      image.save(img_io, 'PNG')
      img_io.seek(0)

      return send_file(img_io, mimetype='image/png')
    except Exception as e:
      return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(port=port, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received text: smile lemon


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2024 13:20:28] "POST /generate HTTP/1.1" 200 -


Received text: smile lemon


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2024 13:20:59] "POST /generate HTTP/1.1" 200 -


In [ ]:
ngrok.disconnect(5000)
ngrok.kill()